In [2]:
import pandas as pd
from sklearn.cluster import KMeans

In [3]:
def rfm_variables(df):
  ## quantidade de vendas
  f_sales = len(df)
  ## quantidade de vendas com lucro
  f_profit = len(df[df['Profit'] > 0])
  ## total de vendas, lucro, quantidade e custo de entrega
  m_sales = round(df['Sales'].sum(),2)
  m_profit = round(df['Profit'].sum(),2)
  m_qty = df['Quantity'].sum()
  m_deliver = round(df['Shipping Cost'].sum(),2)
  ## periodicidade
  df_sort = df[['Order Date']].sort_values(by='Order Date').drop_duplicates()
  df_sort['diff'] = df_sort['Order Date'] - df_sort['Order Date'].shift(1)
  df_sort['diff_int'] = df_sort['diff'].dt.days
  r_days = round(df_sort['diff_int'].mean(),2)
  return f_sales, f_profit, m_sales, m_profit, m_qty, m_deliver, r_days

In [4]:
def fit_data(data, variable):
  rfm = pd.DataFrame()
  variaveis = data[variable].unique()
  for variavel in variaveis:
    dados = data[data[variable] == variavel]
    f_vendas, f_lucro, m_vendas, m_lucro, m_qtde, m_entrega, r_dias = rfm_variables(dados)
    rfm = rfm.append(
        {
            'referencia': variavel, 'm_vendas': m_vendas, 'm_lucro': m_lucro,
            'm_qtde': m_qtde, 'm_entrega': m_entrega, 'r_dias': r_dias,
            'f_vendas': f_vendas, 'f_lucro': f_lucro           
        },
        ignore_index = True
    )
  return rfm

In [6]:
data = pd.read_feather('tavbase/GS.feather')
data

,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,City,State,Country,...,Profit,Shipping Cost,Order Priority,Year,Month,Period,Order Date Month,Delivery,Price,Benefit
0,MX-2014-143658,2014-10-02,2014-10-06,Standard Class,SC-20575,Sonia Cooley,Consumer,Mexico City,Distrito Federal,Mexico,...,4.56,1.033,Medium,2014,10,46,2014-10-01,4,4.36,1
1,MX-2012-155047,2012-10-15,2012-10-20,Standard Class,KW-16570,Kelly Williams,Consumer,Dos Quebradas,Risaralda,Colombia,...,90.72,13.449,Medium,2012,10,22,2012-10-01,5,31.52,1
2,MX-2012-155047,2012-10-15,2012-10-20,Standard Class,KW-16570,Kelly Williams,Consumer,Dos Quebradas,Risaralda,Colombia,...,54.08,9.627,Medium,2012,10,22,2012-10-01,5,96.64,1
3,MX-2012-155047,2012-10-15,2012-10-20,Standard Class,KW-16570,Kelly Williams,Consumer,Dos Quebradas,Risaralda,Colombia,...,4.96,1.371,Medium,2012,10,22,2012-10-01,5,8.86,1
4,MX-2012-155047,2012-10-15,2012-10-20,Standard Class,KW-16570,Kelly Williams,Consumer,Dos Quebradas,Risaralda,Colombia,...,11.44,3.787,Medium,2012,10,22,2012-10-01,5,35.80,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51285,HU-2012-7730,2012-09-05,2012-09-07,First Class,NF-8385,Natalie Fritzler,Consumer,Budapest,Budapest,Hungary,...,3.42,1.980,High,2012,9,21,2012-09-01,2,11.07,1
51286,HU-2012-7730,2012-09-05,2012-09-07,First Class,NF-8385,Natalie Fritzler,Consumer,Budapest,Budapest,Hungary,...,18.42,13.020,High,2012,9,21,2012-09-01,2,30.72,1
51287,HU-2012-7730,2012-09-05,2012-09-07,First Class,NF-8385,Natalie Fritzler,Consumer,Budapest,Budapest,Hungary,...,20.88,8.780,High,2012,9,21,2012-09-01,2,20.13,1
51288,HU-2012-7730,2012-09-05,2012-09-07,First Class,NF-8385,Natalie Fritzler,Consumer,Budapest,Budapest,Hungary,...,33.84,18.970,High,2012,9,21,2012-09-01,2,32.61,1


In [8]:
variaveis = [
    'f_vendas', 'f_lucro', 'm_entrega', 'm_lucro', 'm_qtde', 'm_vendas', 'r_dias'
]

In [13]:
country_rfm = fit_data(data, 'Country')
country_rfm = country_rfm.fillna(0)
country_rfm['cluster'] = KMeans(
    n_clusters=3,
    random_state=0
).fit(
    country_rfm[
        variaveis
    ]
).labels_
country_rfm

/tmp/ipykernel_1883/3273467742.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rfm = rfm.append(
/tmp/ipykernel_1883/3273467742.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rfm = rfm.append(
/tmp/ipykernel_1883/3273467742.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rfm = rfm.append(
/tmp/ipykernel_1883/3273467742.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rfm = rfm.append(
/tmp/ipykernel_1883/3273467742.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rfm = rfm.append(
/tmp/ipykernel_1883/3273467742.py:7: FutureWarning

,referencia,m_vendas,m_lucro,m_qtde,m_entrega,r_dias,f_vendas,f_lucro,cluster
0,Mexico,622590.62,102818.10,10011,67658.40,1.89,2644,2272,2
1,Colombia,81502.53,18798.05,1277,9411.77,9.32,360,350,0
2,Brazil,361106.42,30090.50,6148,38170.04,2.66,1599,1185,2
3,Nicaragua,149687.06,33401.44,2235,17155.17,5.30,614,600,0
4,Panama,51539.93,-17723.45,1426,5416.04,7.78,388,75,0
...,...,...,...,...,...,...,...,...,...
142,Ethiopia,850.62,290.16,18,81.54,358.67,7,7,0
143,Equatorial Guinea,150.51,44.46,3,9.38,0.00,3,3,0
144,Bahrain,669.18,12.84,6,72.76,0.00,2,2,0
145,Chad,1317.03,90.00,11,297.94,0.00,2,1,0


In [21]:
cluster = pd.DataFrame()
for index, row in enumerate(
    KMeans(
        n_clusters=3,
        random_state=0
    ).fit(
        country_rfm[
            variaveis
        ]
    ).cluster_centers_):
    print(row)
    cluster = cluster.append(
        {
            'cluster': index,
            'clf_vendas': row[0],
            'clm_lucro': row[1],
            'clm_entrega': row[2],       
            'clm_lucro': row[3],
            'cl_qtde': row[4],
            'clm_vendas': row[5],
            'clr_dias': row[6]
        },
        ignore_index = True
    )
cluster


[  166.90510949   111.03649635  3728.72131387  2340.3550365
   516.94160584 34488.47510949   111.39007299]
[9.99400000e+03 8.05800000e+03 2.38173790e+05 2.86397020e+05
 3.78730000e+04 2.29720086e+06 1.18000000e+00]
[2.04777778e+03 1.64533333e+03 6.70896756e+04 9.56035122e+04
 7.73533333e+03 6.24486658e+05 2.28555556e+00]


/tmp/ipykernel_1883/3941680971.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cluster = cluster.append(
/tmp/ipykernel_1883/3941680971.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cluster = cluster.append(
/tmp/ipykernel_1883/3941680971.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cluster = cluster.append(


,cluster,clf_vendas,clm_lucro,clm_entrega,cl_qtde,clm_vendas,clr_dias
0,0.0,166.905109,2340.355036,3728.721314,516.941606,3.448848e+04,111.390073
1,1.0,9994.000000,286397.020000,238173.790000,37873.000000,2.297201e+06,1.180000
2,2.0,2047.777778,95603.512222,67089.675556,7735.333333,6.244867e+05,2.285556


In [22]:
country_rfm = country_rfm.merge(
    cluster,
    on='cluster',
    how='left'
)
country_rfm

,referencia,m_vendas,m_lucro,m_qtde,m_entrega,r_dias,f_vendas,f_lucro,cluster,clf_vendas,clm_lucro,clm_entrega,cl_qtde,clm_vendas,clr_dias
0,Mexico,622590.62,102818.10,10011,67658.40,1.89,2644,2272,2,2047.777778,95603.512222,67089.675556,7735.333333,624486.657778,2.285556
1,Colombia,81502.53,18798.05,1277,9411.77,9.32,360,350,0,166.905109,2340.355036,3728.721314,516.941606,34488.475109,111.390073
2,Brazil,361106.42,30090.50,6148,38170.04,2.66,1599,1185,2,2047.777778,95603.512222,67089.675556,7735.333333,624486.657778,2.285556
3,Nicaragua,149687.06,33401.44,2235,17155.17,5.30,614,600,0,166.905109,2340.355036,3728.721314,516.941606,34488.475109,111.390073
4,Panama,51539.93,-17723.45,1426,5416.04,7.78,388,75,0,166.905109,2340.355036,3728.721314,516.941606,34488.475109,111.390073
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142,Ethiopia,850.62,290.16,18,81.54,358.67,7,7,0,166.905109,2340.355036,3728.721314,516.941606,34488.475109,111.390073
143,Equatorial Guinea,150.51,44.46,3,9.38,0.00,3,3,0,166.905109,2340.355036,3728.721314,516.941606,34488.475109,111.390073
144,Bahrain,669.18,12.84,6,72.76,0.00,2,2,0,166.905109,2340.355036,3728.721314,516.941606,34488.475109,111.390073
145,Chad,1317.03,90.00,11,297.94,0.00,2,1,0,166.905109,2340.355036,3728.721314,516.941606,34488.475109,111.390073


In [24]:
country_rfm.to_feather('tavbase/clusterizacao_pais.feather')